In [51]:
import numpy as np
import scipy
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import cvxpy as cvx
from scipy.stats import chisquare, f_oneway, ttest_ind

In [2]:
pendle_borough_data = pd.read_csv('dataset/extracts/pendle_borough_records_extracts.csv', encoding='latin1')
rochdale_borough_data = pd.read_csv('dataset/extracts/rochdale_borough_records_extracts.csv', encoding='latin1')
stockport_metropolitan_borough_data = pd.read_csv('dataset/extracts/stockport_metropolitan_borough_records_extracts.csv', encoding='latin1')

In [3]:
tree = DecisionTreeRegressor()

In [4]:
pendle_borough_data.head()

,supplier_name,value,department,service_description,privilege,trade_cat,service_category
0,British Telecommunications Plc,9000.00,Financial Services,Telephones : Central,Utility,64200000,member
1,BROXAP LIMITED,5424.52,Parks & Recreation Services,Grounds : R & M : Day to Day : Routine,Material Handling,45233293,maintenance
2,Landscape Engineering Ltd,14900.00,Parks & Recreation Services,Grounds : R & M : Day to Day : Routine,Material Handling,45000000,maintenance
3,Landscape Engineering Ltd,14900.00,Parks & Recreation Services,Grounds : R & M : Day to Day : Routine,Material Handling,45000000,maintenance
4,BUSINESS IN THE COMMUNITY,5000.00,Economic Development & Tourism,Miscellaneous,Education,80000000,misc


In [5]:
rochdale_borough_data.head()

,supplier_name,account_name,service,total_value,privilege,trade_cat,service_category
0,ACORN RECOVERY PROJECTS,PH OTHER CONTRACTS,PUBLIC HEALTH,5790.0,Health,85100000,health
1,BARNARDOS,PH BUSINESS CASES,PUBLIC HEALTH,5516.0,Health,85300000,health
2,EARLY BREAK,ACTIVITIES,PUBLIC HEALTH,53913.0,Material,44221000,health
3,EARLY BREAK,ACTIVITIES,PUBLIC HEALTH,49502.0,Social,98000000,health
4,EARLY BREAK,ACTIVITIES,PUBLIC HEALTH,49502.0,Social,98000000,health


In [6]:
input_data = pendle_borough_data.loc[:, ['privilege', 'service_category', 'value']]
privilege_data = input_data.groupby(by=['privilege']).sum()
service_data = input_data.groupby(by=['service_category']).sum()

In [60]:
s = cvx.Variable(service_data.values.shape[0])
p = cvx.Variable(privilege_data.values.shape[0])

data = np.empty((10*service_data.values.shape[0] + 10*privilege_data.values.shape[0],4), dtype=object)

service = cvx.matmul(s, service_data.values[:,0])
privilege = cvx.matmul(p, privilege_data.values[:,0])

dmu_s = np.random.normal(1e30, 10, service_data.values.shape[0])
dmu_p = np.random.normal(1e30, 10, privilege_data.values.shape[0])
# objective function
objective = cvx.Maximize(service)

# constraints
constraints = [cvx.matmul(s, dmu_s) - cvx.matmul(p, dmu_p) <= 0, privilege == 1, s >= 0, p >= 0]

# use cvxpy to solve the objective
problem = cvx.Problem(objective, constraints).solve(verbose=False, solver=cvx.SCS, max_iters=500)

print(dmu_s, s.value)
print(chisquare(dmu_s/dmu_s.max(), s.value/s.value.max()))
print(f_oneway(service_data.values[:,0]/service_data.values.max(), s.value/s.value.max()))
print(ttest_ind(service_data.values[:,0]/service_data.values.max(), s.value/s.value.max()))
for ii,v in enumerate(s.value):
    data[ii] = [i, problem, 'service', v]

print(dmu_p, p.value)
print(chisquare(dmu_p/dmu_p.max(), p.value/p.value.max()))
print(f_oneway(privilege_data.values[:,0]/privilege_data.values.max(), p.value/p.value.max()))
print(ttest_ind(privilege_data.values[:,0]/privilege_data.values.max(), p.value/p.value.max()))
for ij,v in enumerate(p.value):
    data[ij+ii] = [i, problem, 'privilege', v]

# ['data', 'expense', 'finance', 'maintenance', 'member', 'misc']
# ['Administration', 'Data', 'Education', 'Equipment', 'Insurance', 'Material Handling', 'Transport', 'Utility']

[1.e+30 1.e+30 1.e+30 1.e+30 1.e+30 1.e+30] [4.79461104e-07 5.09262979e-07 5.46827181e-07 4.83229230e-07
 4.80661696e-07 4.79291241e-07]
Power_divergenceResult(statistic=0.07174204231374592, pvalue=0.999928522677568)
F_onewayResult(statistic=15.490962524445246, pvalue=0.00279357639768766)
Ttest_indResult(statistic=-3.9358560091097416, pvalue=0.002793576397687649)
[1.e+30 1.e+30 1.e+30 1.e+30 1.e+30 1.e+30 1.e+30 1.e+30] [2.20045759e-07 6.29520979e-07 2.22546177e-07 2.19446274e-07
 2.22001773e-07 1.91454387e-07 1.35436353e-06 2.19984274e-07]
Power_divergenceResult(statistic=27.3165694336249, pvalue=0.00029226028751272025)
F_onewayResult(statistic=0.2307585406824826, pvalue=0.638381436024317)
Ttest_indResult(statistic=-0.48037333469134474, pvalue=0.6383814360243174)


Chi Frequency is higher for service indicating a single exposure does signify for service
F Frequency for higher for privilege indicating double exposures or exposure differences signify for privilege
T Frequency is higher for privilege indicating double exposures signify for privilege

In [9]:
service_data.index, privilege_data.index

(Index(['data', 'expense', 'finance', 'maintenance', 'member', 'misc'], dtype='object', name='service_category'),
 Index(['Administration', 'Data', 'Education', 'Equipment', 'Insurance',
        'Material Handling', 'Transport', 'Utility'],
       dtype='object', name='privilege'))

In [12]:
s.value, p.value

(array([-1.52891295e-10, -1.26642227e-09,  3.41906447e-04, -7.77260629e-10,
        -1.03035447e-09, -1.29933842e-09]),
 array([-1.35087383e-10, -9.17916190e-12, -1.49232396e-10, -1.29841864e-10,
         1.70943084e-04, -5.21692059e-11, -5.61353887e-12, -1.33369569e-10]))

In [10]:
input_data = rochdale_borough_data.loc[:, ['privilege', 'service_category', 'total_value']]
input_data.groupby(by=['privilege', 'service_category']).sum()

total_value
privilege      service_category             
Administration economy              15100.00
               education            10999.40
               environment         115288.73
               health                5423.00
               member               20000.00
...                                      ...
Transport      maintenance          56611.00
               misc                  7280.00
Utility        education            29400.00
               environment         166470.82
               misc                161000.00

[100 rows x 1 columns]